#### -- [INIT] -- 
***

In [22]:
import pygame
from typing import NamedTuple
import os
import sys

class SystemContext(NamedTuple):
    screen :pygame.Surface
    clock :pygame.time.Clock

# System Context will be init on awake
class RenderContext(NamedTuple):
    WIDTH :int
    HEIGHT :int
    BACKGROUND_COLOR :tuple
    FPS :int

class AssetContext(NamedTuple):
    ASSET_FOLDER_PATH :str

# Statically define CONTEXT CONSTANTS
RENDER_CONTEXT = RenderContext(
    640, # RESOLUTION_WIDTH
    480, # RESOLUTION_HEIGHT
    (255, 255, 0), # BACKGROUND_COLOR
    5 # FPS
)

ASSET_CONTEXT = AssetContext(
    ASSET_FOLDER_PATH = os.path.join('assets','images')
)

def on_awake()-> SystemContext:
    ''' 
    Initialize pygame
    - SYS_SCREEN
    - SYS_CLOCK
    '''
    # Initialize pygame
    pygame.init()
    # Initialize the screen
    screen = pygame.display.set_mode(
        # @audit-ok 💨 : Use `HWSURFACE` and `DOUBLEBUF` for better performance
        (RENDER_CONTEXT.WIDTH, RENDER_CONTEXT.HEIGHT),
        pygame.HWSURFACE | pygame.DOUBLEBUF)
    # Initialize the clock
    # @audit-ok 💨 : Cap the frame rate else loop will run at system allows and 
    # # will lead to :
    # - high CPU usage
    # - inconsistent performance across different devices. 
    clock = pygame.time.Clock()

    # Set the title and icon (optional)
    pygame.display.set_caption('Asteroids_ECS_Edition')
    icon = pygame.image.load(
        os.path.join(ASSET_CONTEXT.ASSET_FOLDER_PATH, 'app_icon.jpg')
    ).convert_alpha()
    pygame.display.set_icon(icon)

    return SystemContext(screen, clock)

def on_exit() -> None :
    ''' 
    Handle exit event
    '''
    pygame.quit()
    sys.exit()

ECS Refactor

Entity

In [1]:
import uuid

# Implement a simple Entity class that manages a unique identifier for each entity
class Entity:
    def __init__(self, id=None):
        self.id = id or self.generate_unique_id()

    @staticmethod
    def generate_unique_id():
        return str(uuid.uuid4())

Component

In [2]:
# Create component classes
# These classes should ONLY contain data, NO METHODS
class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Health:
    def __init__(self, health):
        self.health = health

Component Manager

In [3]:
from abc import ABC, abstractmethod
from collections import defaultdict

class ComponentManagerInterface(ABC):
    @abstractmethod
    def add_component(self, entity, component):
        'Add a component to an entity'
        pass

    @abstractmethod
    def remove_component(self, entity, component_type):
        'Remove a component of a specific type from an entity'
        pass

    @abstractmethod
    def get_component(self, entity, component_type):
        'Get a component of a specific type from an entity'
        pass

    @abstractmethod
    def get_entities_with_component(self, component_type):
        'Get a list of entities that have a component of a specific type'
        pass

# Implement a simple ComponentManager class that manages a dictionary of components
class ComponentManager(ComponentManagerInterface):
    def __init__(self):
        # With a `defaultdict`, you can directly add a component without first 
        # checking if the `entity.id` exists in `self.components`. 
        # If the `entity.id` is not present, a new dictionary will be created 
        # automatically :
        # - simplifies the code for `add_component`
        self.components = defaultdict(dict)

    def add_component(self, entity, component):
        # defaultdict will automatically create a new dictionary for the entity
        # if it does not exist : greatly simplifies the code
        self.components[entity.id][type(component)] = component

    def remove_component(self, entity, component_type):
        if entity.id in self.components and component_type in self.components[entity.id]:
            del self.components[entity.id][component_type]

    def get_component(self, entity, component_type):
        return self.components[entity.id].get(component_type, None)

    def get_entities_with_component(self, component_type):
        entities = []
        for entity_id, components in self.components.items():
            if component_type in components:
                entities.append(Entity(entity_id))
        return entities


System


In [27]:
from numpy import random
# Implement Systems that operate on the components
class MovementSystem:
    def update(self, component_manager, delta_time):
        entities = component_manager.get_entities_with_component(Position)
        for entity in entities:
            position = component_manager.get_component(entity, Position)
            position.x += 1 * delta_time
            position.y += 1 * delta_time

class RenderSystem:
    def __init__(self, screen :pygame.Surface, clock :pygame.time.Clock):
        self.screen = screen
        self.clock = clock

    def update(self):
        # Fill the screen with the background color
        rgb_random = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        # self.screen.fill(RENDER_CONTEXT.BACKGROUND_COLOR)
        self.screen.fill(rgb_random)

        # Update the display
        pygame.display.update()

        # Limit the frame rate
        self.clock.tick(RENDER_CONTEXT.FPS)


On_Update

In [28]:
# Initialization
context_sys = on_awake()
component_manager = ComponentManager()
movement_system = MovementSystem()
render_system = RenderSystem(context_sys.screen, context_sys.clock)

# Create entities and components
player = Entity()
player_position = Position(0, 0)
player_health = Health(100)

component_manager.add_component(player, player_position)
component_manager.add_component(player, player_health)

# Main game loop
while True:
    try:
        # Set the frame rate
        context_sys.clock.tick(RENDER_CONTEXT.FPS)
        # Event Handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                on_exit()
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_ESCAPE:
                    on_exit()
        # Update inputs
        # Update positions
        movement_system.update(component_manager, 0.016)
        # Update the display
        render_system.update()
            
    except SystemExit:
        break